 # NN Model on Our Dataset (Keras)

In [1]:
# extraction
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mp  
import csv 
import pandas as pd 
import math
import  cv2


# model
import keras
from keras.utils import to_categorical
from keras.models import Sequential, Input, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

# split and other utils
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Distribution: R = 468, F = 468, L = 468 ; Total = 1404

In [2]:
# Loading dataset
X = np.genfromtxt("Training_Data1.csv", delimiter =',')  
Y = np.genfromtxt("Training_Labels1.csv", delimiter =',')  


In [38]:
#del X_, train_X, train_Y, valid_X, valid_Y

In [22]:
X_ = X/255.0

In [23]:
print(X, "\n", X_, "\n", Y)
print(X.shape, X_.shape, Y.shape)

[[ 28.  55.  54. ...  13.  13.  13.]
 [ 77. 151. 148. ...  21.  21.  21.]
 [ 64. 123.  96. ...  20.  20.  20.]
 ...
 [ 57. 110. 115. ...  42.  42.  42.]
 [ 57. 115. 113. ...  47.  47.  47.]
 [ 53. 105. 105. ...  41.  42.  42.]] 
 [[0.10980392 0.21568627 0.21176471 ... 0.05098039 0.05098039 0.05098039]
 [0.30196078 0.59215686 0.58039216 ... 0.08235294 0.08235294 0.08235294]
 [0.25098039 0.48235294 0.37647059 ... 0.07843137 0.07843137 0.07843137]
 ...
 [0.22352941 0.43137255 0.45098039 ... 0.16470588 0.16470588 0.16470588]
 [0.22352941 0.45098039 0.44313725 ... 0.18431373 0.18431373 0.18431373]
 [0.20784314 0.41176471 0.41176471 ... 0.16078431 0.16470588 0.16470588]] 
 [0. 0. 0. ... 2. 2. 2.]
(1404, 44800) (1404, 44800) (1404,)


In [24]:
train_X, test_X, train_Y, test_Y = train_test_split(X_, Y, test_size=0.2, random_state=13)
print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)

train_X, valid_X, train_Y, valid_Y = train_test_split(train_X, train_Y, test_size=0.2, random_state=13)
print(train_X.shape, valid_X.shape, train_Y.shape, valid_Y.shape)

(1123, 44800) (281, 44800) (1123,) (281,)
(898, 44800) (225, 44800) (898,) (225,)


In [25]:
train_Y = to_categorical(train_Y, num_classes = 3)
valid_Y = to_categorical(valid_Y, num_classes = 3)

In [36]:
# print(train_Y.shape, train_Y)
# print(valid_Y.shape, valid_Y)

In [27]:
row = 140
col = 320

In [29]:
model = Sequential()

# Adding the input layer and the first hidden layer
model.add(Dense(output_dim = 64, init = 'uniform', activation = 'sigmoid', input_dim = row*col))

# # Adding the second hidden layer
# model.add(Dense(output_dim = 32, init = 'uniform', activation = 'sigmoid'))

# Adding the output layer
model.add(Dense(output_dim = 3, init = 'uniform', activation = 'sigmoid'))

epochs = 40
learning_rate = 0.03
decay_rate = learning_rate/epochs
momentum = 0.8
sgd = SGD(lr = learning_rate, momentum = momentum, decay = decay_rate, nesterov=False)

# Compiling the ANN
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", input_dim=44800, units=64, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=3, kernel_initializer="uniform")`
  # Remove the CWD from sys.path while we load stuff.


In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 64)                2867264   
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 195       
Total params: 2,867,459
Trainable params: 2,867,459
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Fitting the ANN to the Training set
model.fit(train_X, train_Y, batch_size = 1, epochs = epochs, validation_data=(valid_X, valid_Y))

Train on 898 samples, validate on 225 samples
Epoch 1/40
898/898 [==============================] - 39s 44ms/step - loss: 1.0910 - categorical_accuracy: 0.4566 - val_loss: 1.2361 - val_categorical_accuracy: 0.3822
Epoch 2/40
898/898 [==============================] - 40s 44ms/step - loss: 1.0862 - categorical_accuracy: 0.4710 - val_loss: 1.3243 - val_categorical_accuracy: 0.3956
Epoch 3/40
898/898 [==============================] - 40s 45ms/step - loss: 1.0655 - categorical_accuracy: 0.4978 - val_loss: 1.1081 - val_categorical_accuracy: 0.4311
Epoch 4/40
898/898 [==============================] - 44s 49ms/step - loss: 1.0343 - categorical_accuracy: 0.5156 - val_loss: 1.0915 - val_categorical_accuracy: 0.4978
Epoch 5/40
898/898 [==============================] - 40s 44ms/step - loss: 1.0234 - categorical_accuracy: 0.5356 - val_loss: 1.1097 - val_categorical_accuracy: 0.5111
Epoch 6/40
898/898 [==============================] - 41s 45ms/step - loss: 1.0344 - categorical_accuracy: 0.5011 

In [32]:
# Evaluate using Test data
test_Y_ = to_categorical(test_Y)
scores = model.evaluate(test_X, test_Y_, verbose = 0)
print("%s: %.2f%%" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.96%
categorical_accuracy: 55.16%


In [35]:
# Predicting the Test Results
pred_y = model.predict(test_X)
print(pred_y.shape,"\n", pred_y)

i=0
pred_Y = np.empty((pred_y.shape[0],1))
for rec in pred_y:
    pred_Y[i] = np.argmax(rec > 0.5)
    i = i+1
    
print(pred_Y.shape)

(281, 3) 
 [[4.26613651e-02 1.38034215e-02 9.43535149e-01]
 [6.53116047e-01 2.92018116e-01 5.48658296e-02]
 [2.65154988e-01 2.10201100e-01 5.24643898e-01]
 [8.34182743e-03 2.35863007e-03 9.89299536e-01]
 [2.63992161e-01 1.24562457e-01 6.11445367e-01]
 [5.27654111e-01 4.43672389e-01 2.86735315e-02]
 [1.96499854e-01 4.43791151e-01 3.59708995e-01]
 [2.45318741e-01 1.42464116e-01 6.12217128e-01]
 [5.20907283e-01 4.20644939e-01 5.84478006e-02]
 [2.32181028e-01 3.02403290e-02 7.37578630e-01]
 [5.01517486e-03 8.57380271e-01 1.37604550e-01]
 [2.47644395e-01 1.36735082e-01 6.15620553e-01]
 [6.75004628e-03 9.41746414e-01 5.15035540e-02]
 [3.55632693e-01 2.80327559e-01 3.64039779e-01]
 [2.35724658e-01 7.04817295e-01 5.94580434e-02]
 [4.59044017e-02 9.49435771e-01 4.65980405e-03]
 [2.11942509e-01 7.31623411e-01 5.64341173e-02]
 [2.47832090e-01 1.36127621e-01 6.16040289e-01]
 [5.60009554e-02 4.29814786e-01 5.14184237e-01]
 [4.75146711e-01 3.43703836e-01 1.81149393e-01]
 [2.46323824e-01 1.36143580e-

In [34]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
 
test_Y = test_Y.reshape((test_Y.shape[0],1))  # to make a 2d matrix
cm = confusion_matrix(test_Y,pred_Y)
print(cm,"\n")

print("Confusion Matrix : \n")
for i in range(3):
    print(np.round(cm[i]/np.sum(cm[i])*100,2))
    #cm[i] = np.round((cm[i]/np.sum(cm[i])) * 100,2)
    
# Performance accuracy 
accuracy = ((test_Y == pred_Y)==True).sum() / len(test_Y) * 100
print("\nAccuracy = "+ str(accuracy) + "%\n")

[[47 22 37]
 [21 52 15]
 [ 9 11 67]] 

Confusion Matrix : 

[44.34 20.75 34.91]
[23.86 59.09 17.05]
[10.34 12.64 77.01]

Accuracy = 59.07473309608541%

